<div style="border: 2px solid #8A9AD0; margin: 1em 0.2em; padding: 0.5em;">

# Advanced SQL

by [The Carpentries](https://training.galaxyproject.org/hall-of-fame/carpentries/), [Helena Rasche](https://training.galaxyproject.org/hall-of-fame/hexylena/), [Avans Hogeschool](https://training.galaxyproject.org/hall-of-fame/avans-atgm/)

CC-BY licensed content from the [Galaxy Training Network](https://training.galaxyproject.org/)

**Objectives**

- How can I calculate sums, averages, and other summary values?
- How can I combine data from multiple tables?
- How should I format data in a database, and why?
- How can I create, modify, and delete tables and data?
- How can I access databases from programs written in Python?

**Objectives**

- Define aggregation and give examples of its use.
- Write queries that compute aggregated values.
- Trace the execution of a query that performs aggregation.
- Explain how missing data is handled during aggregation.
- Explain the operation of a query that joins two tables.
- Explain how to restrict the output of a query containing a join to only include meaningful combinations of values.
- Write queries that join tables on equal keys.
- Explain what primary and foreign keys are, and why they are useful.
- Explain what an atomic value is.
- Distinguish between atomic and non-atomic values.
- Explain why every value in a database should be atomic.
- Explain what a primary key is and why every record should have one.
- Identify primary keys in database tables.
- Explain why database entries should not contain redundant information.
- Identify redundant information in databases.
- Write statements that create tables.
- Write statements to insert, modify, and delete records.
- Write short programs that execute SQL queries.
- Trace the execution of a program that contains an SQL query.
- Explain why most database applications are written in a general-purpose language rather than in SQL.

**Time Estimation: 3H**
</div>


<blockquote class="comment" style="border: 2px solid #ffecc1; margin: 1em 0.2em">
<h3 id="-icon-comment--comment">💬 Comment</h3>
<p>This tutorial is <strong>significantly</strong> based on <a href="https://carpentries.org">the Carpentries</a> <a href="https://github.com/swcarpentry/sql-novice-survey/">Databases and SQL</a> lesson, which is licensed CC-BY 4.0.</p>
<p>Abigail Cabunoc and Sheldon McKay (eds): “Software Carpentry: Using Databases and SQL.”  Version 2017.08, August 2017,
<a href="https://github.com/swcarpentry/sql-novice-survey">github.com/swcarpentry/sql-novice-survey</a>, <a href="https://doi.org/10.5281/zenodo.838776">https://doi.org/10.5281/zenodo.838776</a></p>
<p>Adaptations have been made to make this work better in a GTN/Galaxy environment.</p>
</blockquote>
<blockquote class="agenda" style="border: 2px solid #86D486;display: none; margin: 1em 0.2em">
<h3 id="agenda">Agenda</h3>
<p>In this tutorial, we will cover:</p>
</blockquote>


In [ ]:
# This preamble sets up the sql "magic" for jupyter. Use %%sql in your cells to write sql!
!python3 -m pip install ipython-sql sqlalchemy
!wget -c http://swcarpentry.github.io/sql-novice-survey/files/survey.db
import sqlalchemy
engine = sqlalchemy.create_engine("sqlite:///survey.db")
%load_ext sql
%sql sqlite:///survey.db
%config SqlMagic.displaycon=False

<h1 id="aggregation">Aggregation</h1>
<p>We now want to calculate ranges and averages for our data.
We know how to select all of the dates from the <code style="color: inherit">Visited</code> table:</p>


In [ ]:
%%sql
SELECT dated FROM Visited;

<p>but to combine them,
we must use an aggregation function
such as <code style="color: inherit">min</code> or <code style="color: inherit">max</code>.
Each of these functions takes a set of records as input,
and produces a single record as output:</p>


In [ ]:
%%sql
SELECT min(dated) FROM Visited;

<p><img src="...
dmc+
"" alt="SQL Aggregation" /></p>


In [ ]:
%%sql
SELECT max(dated) FROM Visited;

<p><code class="language-plaintext highlighter-rouge">min</code> and <code style="color: inherit">max</code> are just two of
the aggregation functions built into SQL.
Three others are <code style="color: inherit">avg</code>,
<code class="language-plaintext highlighter-rouge">count</code>,
and <code style="color: inherit">sum</code>:</p>


In [ ]:
%%sql
SELECT avg(reading) FROM Survey WHERE quant = 'sal';

In [ ]:
%%sql
SELECT count(reading) FROM Survey WHERE quant = 'sal';

In [ ]:
%%sql
SELECT sum(reading) FROM Survey WHERE quant = 'sal';

<p>We used <code style="color: inherit">count(reading)</code> here,
but we could just as easily have counted <code style="color: inherit">quant</code>
or any other field in the table,
or even used <code style="color: inherit">count(*)</code>,
since the function doesn’t care about the values themselves,
just how many values there are.</p>
<p>SQL lets us do several aggregations at once.
We can,
for example,
find the range of sensible salinity measurements:</p>


In [ ]:
%%sql
SELECT min(reading), max(reading) FROM Survey WHERE quant = 'sal' AND reading <= 1.0;

<p>We can also combine aggregated results with raw results,
although the output might surprise you:</p>


In [ ]:
%%sql
SELECT person, count(*) FROM Survey WHERE quant = 'sal' AND reading <= 1.0;

<p>Why does Lake’s name appear rather than Roerich’s or Dyer’s?
The answer is that when it has to aggregate a field,
but isn’t told how to,
the database manager chooses an actual value from the input set.
It might use the first one processed,
the last one,
or something else entirely.</p>
<p>Another important fact is that when there are no values to aggregate —
for example, where there are no rows satisfying the <code style="color: inherit">WHERE</code> clause —
aggregation’s result is “don’t know”
rather than zero or some other arbitrary value:</p>


In [ ]:
%%sql
SELECT person, max(reading), sum(reading) FROM Survey WHERE quant = 'missing';

<p>One final important feature of aggregation functions is that
they are inconsistent with the rest of SQL in a very useful way.
If we add two values,
and one of them is null,
the result is null.
By extension,
if we use <code style="color: inherit">sum</code> to add all the values in a set,
and any of those values are null,
the result should also be null.
It’s much more useful,
though,
for aggregation functions to ignore null values
and only combine those that are non-null.
This behavior lets us write our queries as:</p>


In [ ]:
%%sql
SELECT min(dated) FROM Visited;

<p>instead of always having to filter explicitly:</p>


In [ ]:
%%sql
SELECT min(dated) FROM Visited WHERE dated IS NOT NULL;

<p>Aggregating all records at once doesn’t always make sense.
For example,
suppose we suspect that there is a systematic bias in our data,
and that some scientists’ radiation readings are higher than others.
We know that this doesn’t work:</p>


In [ ]:
%%sql
SELECT person, count(reading), round(avg(reading), 2)
FROM  Survey
WHERE quant = 'rad';

<p>because the database manager selects a single arbitrary scientist’s name
rather than aggregating separately for each scientist.
Since there are only five scientists,
we could write five queries of the form:</p>


In [ ]:
%%sql
SELECT person, count(reading), round(avg(reading), 2)
FROM  Survey
WHERE quant = 'rad'
AND   person = 'dyer';

<p>but this would be tedious,
and if we ever had a data set with fifty or five hundred scientists,
the chances of us getting all of those queries right is small.</p>
<p>What we need to do is
tell the database manager to aggregate the hours for each scientist separately
using a <code style="color: inherit">GROUP BY</code> clause:</p>


In [ ]:
%%sql
SELECT   person, count(reading), round(avg(reading), 2)
FROM     Survey
WHERE    quant = 'rad'
GROUP BY person;

<p><code class="language-plaintext highlighter-rouge">GROUP BY</code> does exactly what its name implies:
groups all the records with the same value for the specified field together
so that aggregation can process each batch separately.
Since all the records in each batch have the same value for <code style="color: inherit">person</code>,
it no longer matters that the database manager
is picking an arbitrary one to display
alongside the aggregated <code style="color: inherit">reading</code> values.</p>
<blockquote class="tip" style="border: 2px solid #FFE19E; margin: 1em 0.2em">
<h3 id="-icon-tip--know-excel-its-just-a-pivot-table">💡 Know Excel? It’s just a pivot table.</h3>
<p><code style="color: inherit">GROUP BY</code> is basically just a pivot table for Excel users, it lets you build
nice summary tables which aggregate your results.</p>
<p>And if you didn’t already know the Excel equivalent, now you know what to
look for when you need it!</p>
</blockquote>
<p>Just as we can sort by multiple criteria at once,
we can also group by multiple criteria.
To get the average reading by scientist and quantity measured,
for example,
we just add another field to the <code style="color: inherit">GROUP BY</code> clause:</p>


In [ ]:
%%sql
SELECT   person, quant, count(reading), round(avg(reading), 2)
FROM     Survey
GROUP BY person, quant;

<p>Note that we have added <code style="color: inherit">quant</code> to the list of fields displayed,
since the results wouldn’t make much sense otherwise.</p>
<p>Let’s go one step further and remove all the entries
where we don’t know who took the measurement:</p>


In [ ]:
%%sql
SELECT   person, quant, count(reading), round(avg(reading), 2)
FROM     Survey
WHERE    person IS NOT NULL
GROUP BY person, quant
ORDER BY person, quant;

<p>Looking more closely,
this query:</p>
<ol>
<li>selected records from the <code style="color: inherit">Survey</code> table where the <code style="color: inherit">person</code> field was not null;</li>
<li>grouped those records into subsets so that the <code style="color: inherit">person</code> and <code style="color: inherit">quant</code> values in each subset were the same;</li>
<li>ordered those subsets first by <code style="color: inherit">person</code>, and then within each sub-group by <code style="color: inherit">quant</code>; and</li>
<li>counted the number of records in each subset, calculated the average <code style="color: inherit">reading</code> in each, and chose a <code style="color: inherit">person</code> and <code style="color: inherit">quant</code> value from each (it doesn’t matter which ones, since they’re all equal).</li>
</ol>
<blockquote class="question" style="border: 2px solid #8A9AD0; margin: 1em 0.2em">
<h3 id="-icon-question--question-counting-temperature-readings">❓ Question: Counting Temperature Readings</h3>
<p>How many temperature readings did Frank Pabodie record,
and what was their average value?</p>
<blockquote class="solution" style="display:none">
<div style="color: #555; font-size: 95%;">Hint: Select the text with your mouse to see the answer</div><h3 id="-icon-solution--solution">👁 Solution</h3>
<div class="language-plaintext highlighter-rouge"><div><pre style="color: inherit; background: transparent"><code style="color: inherit">SELECT count(reading), avg(reading) FROM Survey WHERE quant = 'temp' AND person = 'pb';
</code></pre></div>    </div>
<table>
<thead>
<tr>
<th>count(reading)</th>
<th>avg(reading)</th>
</tr>
</thead>
<tbody>
<tr>
<td>2</td>
<td>-20.0</td>
</tr>
</tbody>
</table>
</blockquote>
</blockquote>


In [ ]:
%%sql
-- Try solutions here!

<blockquote class="question" style="border: 2px solid #8A9AD0; margin: 1em 0.2em">
<h2 id="averaging-with-null">Averaging with NULL</h2>
<p>The average of a set of values is the sum of the values
divided by the number of values.
Does this mean that the <code style="color: inherit">avg</code> function returns 2.0 or 3.0
when given the values 1.0, <code style="color: inherit">null</code>, and 5.0?</p>
<blockquote class="solution" style="display:none">
<div style="color: #555; font-size: 95%;">Hint: Select the text with your mouse to see the answer</div><h3 id="-icon-solution--solution">👁 Solution</h3>
<p>The answer is 3.0.
<code style="color: inherit">NULL</code> is not a value; it is the absence of a value.
As such it is not included in the calculation.</p>
<p>You can confirm this, by executing this code:</p>
<div class="language-plaintext highlighter-rouge"><div><pre style="color: inherit; background: transparent"><code style="color: inherit">SELECT AVG(a) FROM (
    SELECT 1 AS a
    UNION ALL SELECT NULL
    UNION ALL SELECT 5);
</code></pre></div>    </div>
</blockquote>
</blockquote>


In [ ]:
%%sql
-- Try solutions here!

<blockquote class="question" style="border: 2px solid #8A9AD0; margin: 1em 0.2em">
<h3 id="-icon-question--question-what-does-this-query-do">❓ Question: What Does This Query Do?</h3>
<p>We want to calculate the difference between
each individual radiation reading
and the average of all the radiation readings.
We write the query:</p>
<div class="language-plaintext highlighter-rouge"><div><pre style="color: inherit; background: transparent"><code style="color: inherit">SELECT reading - avg(reading) FROM Survey WHERE quant = 'rad';
</code></pre></div>  </div>
<p>What does this actually produce, and can you think of why?</p>
<blockquote class="solution" style="display:none">
<div style="color: #555; font-size: 95%;">Hint: Select the text with your mouse to see the answer</div><h3 id="-icon-solution--solution">👁 Solution</h3>
<p>The query produces only one row of results when we what we really want is a result for each of the readings.
The <code style="color: inherit">avg()</code> function produces only a single value, and because it is run first, the table is reduced to a single row.
The <code style="color: inherit">reading</code> value is simply an arbitrary one.</p>
<p>To achieve what we wanted, we would have to run two queries:</p>
<div class="language-plaintext highlighter-rouge"><div><pre style="color: inherit; background: transparent"><code style="color: inherit">SELECT avg(reading) FROM Survey WHERE quant='rad';
</code></pre></div>    </div>
<p>This produces the average value (6.5625), which we can then insert into a second query:</p>
<div class="language-plaintext highlighter-rouge"><div><pre style="color: inherit; background: transparent"><code style="color: inherit">SELECT reading - 6.5625 FROM Survey WHERE quant = 'rad';
</code></pre></div>    </div>
<p>This produces what we want, but we can combine this into a single query using subqueries.</p>
<div class="language-plaintext highlighter-rouge"><div><pre style="color: inherit; background: transparent"><code style="color: inherit">SELECT reading - (SELECT avg(reading) FROM Survey WHERE quant='rad') FROM Survey WHERE quant = 'rad';
</code></pre></div>    </div>
<p>This way we don’t have execute two queries.</p>
<p>In summary what we have done is to replace <code style="color: inherit">avg(reading)</code> with <code style="color: inherit">(SELECT avg(reading) FROM Survey WHERE quant='rad')</code> in the original query.</p>
</blockquote>
</blockquote>


In [ ]:
%%sql
-- Try solutions here!

<blockquote class="question" style="border: 2px solid #8A9AD0; margin: 1em 0.2em">
<h3 id="-icon-question--question-ordering-when-concatenating">❓ Question: Ordering When Concatenating</h3>
<p>The function <code style="color: inherit">group_concat(field, separator)</code>
concatenates all the values in a field
using the specified separator character
(or ‘,’ if the separator isn’t specified).
Use this to produce a one-line list of scientists’ names,
such as:</p>
<div class="language-plaintext highlighter-rouge"><div><pre style="color: inherit; background: transparent"><code style="color: inherit">William Dyer, Frank Pabodie, Anderson Lake, Valentina Roerich, Frank Danforth
</code></pre></div>  </div>
<p>Can you find a way to order the list by surname?</p>
</blockquote>


In [ ]:
%%sql
-- Try solutions here!

<h1 id="combining-data">Combining Data</h1>
<p>In order to submit our data to a web site
that aggregates historical meteorological data,
we might need to format it as
latitude, longitude, date, quantity, and reading.
However,
our latitudes and longitudes are in the <code style="color: inherit">Site</code> table,
while the dates of measurements are in the <code style="color: inherit">Visited</code> table
and the readings themselves are in the <code style="color: inherit">Survey</code> table.
We need to combine these tables somehow.</p>
<p>This figure shows the relations between the tables:</p>
<p><img src="data:image/svg+xml;base64,PD94bWwgdmVyc2lvbj0iMS4wIiBlbmNvZGluZz0iVVRGLTgiIHN0YW5kYWxv
bmU9Im5vIj8+CjwhRE9DVFlQRSBzdmcgUFVCTElDICItLy9XM0MvL0RURCBT
VkcgMS4wLy9FTiIKICJodHRwOi8vd3d3LnczLm9yZy9UUi8yMDAxL1JFQy1T
VkctMjAwMTA5MDQvRFREL3N2ZzEwLmR0ZCIgWwogPCFBVFRMSVNUIHN2ZyB4
bWxuczp4bGluayBDREFUQSAjRklYRUQgImh0dHA6Ly93d3cudzMub3JnLzE5
OTkveGxpbmsiPgpdPgo8IS0tIEdlbmVyYXRlZCBieSBHcmFwaHZpeiB2ZXJz
aW9uIDIuMTIgKE1vbiBEZWMgIDQgMjI6MDQ6MzcgVVRDIDIwMDYpCiAgICAg
Rm9yIHVzZXI6IE5pY2sgSiAtLT4KPCEtLSBUaXRsZTogRyBQYWdlczogMSAt
LT4KPHN2ZyB3aWR0aD0iMy44OGluIiBoZWlnaHQ9IjUuNDRpbiIKIHhtbG5z
PSJodHRwOi8vd3d3LnczLm9yZy8yMDAwL3N2ZyIgeG1sbnM6eGxpbms9Imh0
dHA6Ly93d3cudzMub3JnLzE5OTkveGxpbmsiPgo8ZyBpZD0iZ3JhcGgwIiBj
bGFzcz0iZ3JhcGgiIHRyYW5zZm9ybT0ic2NhbGUoMS4zMzMzMyAxLjMzMzMz
KSByb3RhdGUoMCkgdHJhbnNsYXRlKDQgMzg4KSI+Cjx0aXRsZT5HPC90aXRs
ZT4KPHBvbHlnb24gc3R5bGU9ImZpbGw6d2hpdGU7c3Ryb2tlOndoaXRlOyIg
cG9pbnRzPSItNCw0IC00LC0zODggMjc1LC0zODggMjc1LDQgLTQsNCIvPgo8
IS0tIFBlcnNvbiAtLT4KPGcgaWQ9Im5vZGUxIiBjbGFzcz0ibm9kZSI+PHRp
dGxlPlBlcnNvbjwvdGl0bGU+Cjxwb2x5Z29uIHN0eWxlPSJmaWxsOm5vbmU7
c3Ryb2tlOmJsYWNrOyIgcG9pbnRzPSIwLC0xNTYgMCwtMjUyIDEwMiwtMjUy
IDEwMiwtMTU2IDAsLTE1NiIvPgo8dGV4dCB0ZXh0LWFuY2hvcj0ibWlkZGxl
IiB4PSI1MSIgeT0iLTIzNCIgPlBlcnNvbjwvdGV4dD4KPHBvbHlsaW5lIHN0
eWxlPSJmaWxsOm5vbmU7c3Ryb2tlOmJsYWNrOyIgcG9pbnRzPSIwLC0yMjgg
MTAyLC0yMjggIi8+Cjx0ZXh0IHRleHQtYW5jaG9yPSJtaWRkbGUiIHg9IjI3
IiB5PSItMjEwIiA+aWQ8L3RleHQ+Cjxwb2x5bGluZSBzdHlsZT0iZmlsbDpu
b25lO3N0cm9rZTpibGFjazsiIHBvaW50cz0iNTQsLTIwNCA1NCwtMjI4ICIv
Pgo8dGV4dCB0ZXh0LWFuY2hvcj0ibWlkZGxlIiB4PSI3OCIgeT0iLTIxMCIg
PnRleHQ8L3RleHQ+Cjxwb2x5bGluZSBzdHlsZT0iZmlsbDpub25lO3N0cm9r
ZTpibGFjazsiIHBvaW50cz0iMCwtMjA0IDEwMiwtMjA0ICIvPgo8dGV4dCB0
ZXh0LWFuY2hvcj0ibWlkZGxlIiB4PSIzMiIgeT0iLTE4NiIgPnBlcnNvbmFs
PC90ZXh0Pgo8cG9seWxpbmUgc3R5bGU9ImZpbGw6bm9uZTtzdHJva2U6Ymxh
Y2s7IiBwb2ludHM9IjY0LC0xODAgNjQsLTIwNCAiLz4KPHRleHQgdGV4dC1h
bmNob3I9Im1pZGRsZSIgeD0iODMiIHk9Ii0xODYiID50ZXh0PC90ZXh0Pgo8
cG9seWxpbmUgc3R5bGU9ImZpbGw6bm9uZTtzdHJva2U6YmxhY2s7IiBwb2lu
dHM9IjAsLTE4MCAxMDIsLTE4MCAiLz4KPHRleHQgdGV4dC1hbmNob3I9Im1p
ZGRsZSIgeD0iMjgiIHk9Ii0xNjIiID5mYW1pbHk8L3RleHQ+Cjxwb2x5bGlu
ZSBzdHlsZT0iZmlsbDpub25lO3N0cm9rZTpibGFjazsiIHBvaW50cz0iNTcs
LTE1NiA1NywtMTgwICIvPgo8dGV4dCB0ZXh0LWFuY2hvcj0ibWlkZGxlIiB4
PSI3OSIgeT0iLTE2MiIgPnRleHQ8L3RleHQ+CjwvZz4KPCEtLSBTdXJ2ZXkg
LS0+CjxnIGlkPSJub2RlNCIgY2xhc3M9Im5vZGUiPjx0aXRsZT5TdXJ2ZXk8
L3RpdGxlPgo8cG9seWdvbiBzdHlsZT0iZmlsbDpub25lO3N0cm9rZTpibGFj
azsiIHBvaW50cz0iMTY5LDAgMTY5LC0xMjAgMjcyLC0xMjAgMjcyLDAgMTY5
LDAiLz4KPHRleHQgdGV4dC1hbmNob3I9Im1pZGRsZSIgeD0iMjIwIiB5PSIt
MTAyIiA+U3VydmV5PC90ZXh0Pgo8cG9seWxpbmUgc3R5bGU9ImZpbGw6bm9u
ZTtzdHJva2U6YmxhY2s7IiBwb2ludHM9IjE2OSwtOTYgMjcyLC05NiAiLz4K
PHRleHQgdGV4dC1hbmNob3I9Im1pZGRsZSIgeD0iMTkzIiB5PSItNzgiID50
YWtlbjwvdGV4dD4KPHBvbHlsaW5lIHN0eWxlPSJmaWxsOm5vbmU7c3Ryb2tl
OmJsYWNrOyIgcG9pbnRzPSIyMTcsLTcyIDIxNywtOTYgIi8+Cjx0ZXh0IHRl
eHQtYW5jaG9yPSJtaWRkbGUiIHg9IjI0NCIgeT0iLTc4IiA+aW50ZWdlcjwv
dGV4dD4KPHBvbHlsaW5lIHN0eWxlPSJmaWxsOm5vbmU7c3Ryb2tlOmJsYWNr
OyIgcG9pbnRzPSIxNjksLTcyIDI3MiwtNzIgIi8+Cjx0ZXh0IHRleHQtYW5j
aG9yPSJtaWRkbGUiIHg9IjE5OSIgeT0iLTU0IiA+cGVyc29uPC90ZXh0Pgo8
cG9seWxpbmUgc3R5bGU9ImZpbGw6bm9uZTtzdHJva2U6YmxhY2s7IiBwb2lu
dHM9IjIyOSwtNDggMjI5LC03MiAiLz4KPHRleHQgdGV4dC1hbmNob3I9Im1p
ZGRsZSIgeD0iMjUwIiB5PSItNTQiID50ZXh0PC90ZXh0Pgo8cG9seWxpbmUg
c3R5bGU9ImZpbGw6bm9uZTtzdHJva2U6YmxhY2s7IiBwb2ludHM9IjE2OSwt
NDggMjcyLC00OCAiLz4KPHRleHQgdGV4dC1hbmNob3I9Im1pZGRsZSIgeD0i
MTk3IiB5PSItMzAiID5xdWFudDwvdGV4dD4KPHBvbHlsaW5lIHN0eWxlPSJm
aWxsOm5vbmU7c3Ryb2tlOmJsYWNrOyIgcG9pbnRzPSIyMjUsLTI0IDIyNSwt
NDggIi8+Cjx0ZXh0IHRleHQtYW5jaG9yPSJtaWRkbGUiIHg9IjI0OCIgeT0i
LTMwIiA+dGV4dDwvdGV4dD4KPHBvbHlsaW5lIHN0eWxlPSJmaWxsOm5vbmU7
c3Ryb2tlOmJsYWNrOyIgcG9pbnRzPSIxNjksLTI0IDI3MiwtMjQgIi8+Cjx0
ZXh0IHRleHQtYW5jaG9yPSJtaWRkbGUiIHg9IjIwMCIgeT0iLTYiID5yZWFk
aW5nPC90ZXh0Pgo8cG9seWxpbmUgc3R5bGU9ImZpbGw6bm9uZTtzdHJva2U6
YmxhY2s7IiBwb2ludHM9IjIzMSwwIDIzMSwtMjQgIi8+Cjx0ZXh0IHRleHQt
YW5jaG9yPSJtaWRkbGUiIHg9IjI1MSIgeT0iLTYiID5yZWFsPC90ZXh0Pgo8
L2c+CjwhLS0gUGVyc29uJiM0NTsmZ3Q7U3VydmV5IC0tPgo8ZyBpZD0iZWRn
ZTYiIGNsYXNzPSJlZGdlIj48dGl0bGU+UGVyc29uJiM0NTsmZ3Q7U3VydmV5
PC90aXRsZT4KPHBhdGggc3R5bGU9ImZpbGw6bm9uZTtzdHJva2U6YmxhY2s7
IiBkPSJNMTAyLC0yMTZDMTMwLC0yMTYgMTEwLC0xODEgMTIwLC0xNTYgMTM3
LC0xMTUgMTI0LC02OCAxNTksLTYxIi8+Cjxwb2x5Z29uIHN0eWxlPSJmaWxs
OmJsYWNrO3N0cm9rZTpibGFjazsiIHBvaW50cz0iMTU5LjM5OCwtNjQuNDc3
OCAxNjksLTYwIDE1OC43MDEsLTU3LjUxMjUgMTU5LjM5OCwtNjQuNDc3OCIv
Pgo8L2c+CjwhLS0gU2l0ZSAtLT4KPGcgaWQ9Im5vZGUyIiBjbGFzcz0ibm9k
ZSI+PHRpdGxlPlNpdGU8L3RpdGxlPgo8cG9seWdvbiBzdHlsZT0iZmlsbDpu
b25lO3N0cm9rZTpibGFjazsiIHBvaW50cz0iNTMsLTI4OCA1MywtMzg0IDEz
OCwtMzg0IDEzOCwtMjg4IDUzLC0yODgiLz4KPHRleHQgdGV4dC1hbmNob3I9
Im1pZGRsZSIgeD0iOTUiIHk9Ii0zNjYiID5TaXRlPC90ZXh0Pgo8cG9seWxp
bmUgc3R5bGU9ImZpbGw6bm9uZTtzdHJva2U6YmxhY2s7IiBwb2ludHM9IjUz
LC0zNjAgMTM4LC0zNjAgIi8+Cjx0ZXh0IHRleHQtYW5jaG9yPSJtaWRkbGUi
IHg9Ijc2IiB5PSItMzQyIiA+bmFtZTwvdGV4dD4KPHBvbHlsaW5lIHN0eWxl
PSJmaWxsOm5vbmU7c3Ryb2tlOmJsYWNrOyIgcG9pbnRzPSIxMDAsLTMzNiAx
MDAsLTM2MCAiLz4KPHRleHQgdGV4dC1hbmNob3I9Im1pZGRsZSIgeD0iMTE5
IiB5PSItMzQyIiA+dGV4dDwvdGV4dD4KPHBvbHlsaW5lIHN0eWxlPSJmaWxs
Om5vbmU7c3Ryb2tlOmJsYWNrOyIgcG9pbnRzPSI1MywtMzM2IDEzOCwtMzM2
ICIvPgo8dGV4dCB0ZXh0LWFuY2hvcj0ibWlkZGxlIiB4PSI3MiIgeT0iLTMx
OCIgPmxhdDwvdGV4dD4KPHBvbHlsaW5lIHN0eWxlPSJmaWxsOm5vbmU7c3Ry
b2tlOmJsYWNrOyIgcG9pbnRzPSI5MSwtMzEyIDkxLC0zMzYgIi8+Cjx0ZXh0
IHRleHQtYW5jaG9yPSJtaWRkbGUiIHg9IjExNCIgeT0iLTMxOCIgPnJlYWw8
L3RleHQ+Cjxwb2x5bGluZSBzdHlsZT0iZmlsbDpub25lO3N0cm9rZTpibGFj
azsiIHBvaW50cz0iNTMsLTMxMiAxMzgsLTMxMiAiLz4KPHRleHQgdGV4dC1h
bmNob3I9Im1pZGRsZSIgeD0iNzUiIHk9Ii0yOTQiID5sb25nPC90ZXh0Pgo8
cG9seWxpbmUgc3R5bGU9ImZpbGw6bm9uZTtzdHJva2U6YmxhY2s7IiBwb2lu
dHM9Ijk3LC0yODggOTcsLTMxMiAiLz4KPHRleHQgdGV4dC1hbmNob3I9Im1p
ZGRsZSIgeD0iMTE3IiB5PSItMjk0IiA+cmVhbDwvdGV4dD4KPC9nPgo8IS0t
IFZpc2l0ZWQgLS0+CjxnIGlkPSJub2RlMyIgY2xhc3M9Im5vZGUiPjx0aXRs
ZT5WaXNpdGVkPC90aXRsZT4KPHBvbHlnb24gc3R5bGU9ImZpbGw6bm9uZTtz
dHJva2U6YmxhY2s7IiBwb2ludHM9IjEzOCwtMTU2IDEzOCwtMjUyIDIzNywt
MjUyIDIzNywtMTU2IDEzOCwtMTU2Ii8+Cjx0ZXh0IHRleHQtYW5jaG9yPSJt
aWRkbGUiIHg9IjE4NyIgeT0iLTIzNCIgPlZpc2l0ZWQ8L3RleHQ+Cjxwb2x5
bGluZSBzdHlsZT0iZmlsbDpub25lO3N0cm9rZTpibGFjazsiIHBvaW50cz0i
MTM4LC0yMjggMjM3LC0yMjggIi8+Cjx0ZXh0IHRleHQtYW5jaG9yPSJtaWRk
bGUiIHg9IjE2MCIgeT0iLTIxMCIgPmlkPC90ZXh0Pgo8cG9seWxpbmUgc3R5
bGU9ImZpbGw6bm9uZTtzdHJva2U6YmxhY2s7IiBwb2ludHM9IjE4MiwtMjA0
IDE4MiwtMjI4ICIvPgo8dGV4dCB0ZXh0LWFuY2hvcj0ibWlkZGxlIiB4PSIy
MDkiIHk9Ii0yMTAiID5pbnRlZ2VyPC90ZXh0Pgo8cG9seWxpbmUgc3R5bGU9
ImZpbGw6bm9uZTtzdHJva2U6YmxhY2s7IiBwb2ludHM9IjEzOCwtMjA0IDIz
NywtMjA0ICIvPgo8dGV4dCB0ZXh0LWFuY2hvcj0ibWlkZGxlIiB4PSIxNjIi
IHk9Ii0xODYiID5zaXRlPC90ZXh0Pgo8cG9seWxpbmUgc3R5bGU9ImZpbGw6
bm9uZTtzdHJva2U6YmxhY2s7IiBwb2ludHM9IjE4NiwtMTgwIDE4NiwtMjA0
ICIvPgo8dGV4dCB0ZXh0LWFuY2hvcj0ibWlkZGxlIiB4PSIyMTEiIHk9Ii0x
ODYiID50ZXh0PC90ZXh0Pgo8cG9seWxpbmUgc3R5bGU9ImZpbGw6bm9uZTtz
dHJva2U6YmxhY2s7IiBwb2ludHM9IjEzOCwtMTgwIDIzNywtMTgwICIvPgo8
dGV4dCB0ZXh0LWFuY2hvcj0ibWlkZGxlIiB4PSIxNjUiIHk9Ii0xNjIiID5k
YXRlZDwvdGV4dD4KPHBvbHlsaW5lIHN0eWxlPSJmaWxsOm5vbmU7c3Ryb2tl
OmJsYWNrOyIgcG9pbnRzPSIxOTIsLTE1NiAxOTIsLTE4MCAiLz4KPHRleHQg
dGV4dC1hbmNob3I9Im1pZGRsZSIgeD0iMjE0IiB5PSItMTYyIiA+dGV4dDwv
dGV4dD4KPC9nPgo8IS0tIFNpdGUmIzQ1OyZndDtWaXNpdGVkIC0tPgo8ZyBp
ZD0iZWRnZTIiIGNsYXNzPSJlZGdlIj48dGl0bGU+U2l0ZSYjNDU7Jmd0O1Zp
c2l0ZWQ8L3RpdGxlPgo8cGF0aCBzdHlsZT0iZmlsbDpub25lO3N0cm9rZTpi
bGFjazsiIGQ9Ik0xMzgsLTM0OEMxNjgsLTM0OCAxMTksLTIyNiAxMzAsLTE5
OCIvPgo8cG9seWdvbiBzdHlsZT0iZmlsbDpibGFjaztzdHJva2U6YmxhY2s7
IiBwb2ludHM9IjEzMi4xLC0yMDAuOCAxMzgsLTE5MiAxMjcuOSwtMTk1LjIg
MTMyLjEsLTIwMC44Ii8+CjwvZz4KPCEtLSBWaXNpdGVkJiM0NTsmZ3Q7U3Vy
dmV5IC0tPgo8ZyBpZD0iZWRnZTQiIGNsYXNzPSJlZGdlIj48dGl0bGU+Vmlz
aXRlZCYjNDU7Jmd0O1N1cnZleTwvdGl0bGU+CjxwYXRoIHN0eWxlPSJmaWxs
Om5vbmU7c3Ryb2tlOmJsYWNrOyIgZD0iTTIzNywtMjE2QzI2MywtMjE2IDI1
MCwtMTc4IDIzNywtMTU2IDIxOSwtMTI2IDE4NiwtMTQ5IDE2OSwtMTIwIDE2
MywtMTEwIDE1NywtOTYgMTYwLC04OSIvPgo8cG9seWdvbiBzdHlsZT0iZmls
bDpibGFjaztzdHJva2U6YmxhY2s7IiBwb2ludHM9IjE2MS45NTgsLTkxLjkx
NiAxNjksLTg0IDE1OC41NTksLTg1Ljc5NjkgMTYxLjk1OCwtOTEuOTE2Ii8+
CjwvZz4KPC9nPgo8L3N2Zz4K
"" alt="Survey Database Structure" /></p>
<p>The SQL command to do this is <code style="color: inherit">JOIN</code>.
To see how it works,
let’s start by joining the <code style="color: inherit">Site</code> and <code style="color: inherit">Visited</code> tables:</p>


In [ ]:
%%sql
SELECT * FROM Site JOIN Visited;

<p><code class="language-plaintext highlighter-rouge">JOIN</code> creates
the cross product
of two tables,
i.e.,
it joins each record of one table with each record of the other table
to give all possible combinations.
Since there are three records in <code style="color: inherit">Site</code>
and eight in <code style="color: inherit">Visited</code>,
the join’s output has 24 records (3 * 8 = 24) .
And since each table has three fields,
the output has six fields (3 + 3 = 6).</p>
<p>What the join <em>hasn’t</em> done is
figure out if the records being joined have anything to do with each other.
It has no way of knowing whether they do or not until we tell it how.
To do that,
we add a clause specifying that
we’re only interested in combinations that have the same site name,
thus we need to use a filter:</p>


In [ ]:
%%sql
SELECT * FROM Site JOIN Visited ON Site.name = Visited.site;

<p><code class="language-plaintext highlighter-rouge">ON</code> is very similar to <code style="color: inherit">WHERE</code>,
and for all the queries in this lesson you can use them interchangeably.
There are differences in how they affect [outer joins][outer],
but that’s beyond the scope of this lesson.
Once we add this to our query,
the database manager throws away records
that combined information about two different sites,
leaving us with just the ones we want.</p>
<p>Notice that we used <code style="color: inherit">Table.field</code> to specify field names
in the output of the join.
We do this because tables can have fields with the same name,
and we need to be specific which ones we’re talking about.
For example,
if we joined the <code style="color: inherit">Person</code> and <code style="color: inherit">Visited</code> tables,
the result would inherit a field called <code style="color: inherit">id</code>
from each of the original tables.</p>
<p>We can now use the same dotted notation
to select the three columns we actually want
out of our join:</p>


In [ ]:
%%sql
SELECT Site.lat, Site.long, Visited.dated
FROM   Site JOIN Visited
ON     Site.name = Visited.site;

<p>If joining two tables is good,
joining many tables must be better.
In fact,
we can join any number of tables
simply by adding more <code style="color: inherit">JOIN</code> clauses to our query,
and more <code style="color: inherit">ON</code> tests to filter out combinations of records
that don’t make sense:</p>


In [ ]:
%%sql
SELECT Site.lat, Site.long, Visited.dated, Survey.quant, Survey.reading
FROM   Site JOIN Visited JOIN Survey
ON     Site.name = Visited.site
AND    Visited.id = Survey.taken
AND    Visited.dated IS NOT NULL;

<p>We can tell which records from <code style="color: inherit">Site</code>, <code style="color: inherit">Visited</code>, and <code style="color: inherit">Survey</code>
correspond with each other
because those tables contain
primary keys
and foreign keys.
A primary key is a value,
or combination of values,
that uniquely identifies each record in a table.
A foreign key is a value (or combination of values) from one table
that identifies a unique record in another table.
Another way of saying this is that
a foreign key is the primary key of one table
that appears in some other table.
In our database,
<code style="color: inherit">Person.id</code> is the primary key in the <code style="color: inherit">Person</code> table,
while <code style="color: inherit">Survey.person</code> is a foreign key
relating the <code style="color: inherit">Survey</code> table’s entries
to entries in <code style="color: inherit">Person</code>.</p>
<p>Most database designers believe that
every table should have a well-defined primary key.
They also believe that this key should be separate from the data itself,
so that if we ever need to change the data,
we only need to make one change in one place.
One easy way to do this is
to create an arbitrary, unique ID for each record
as we add it to the database.
This is actually very common:
those IDs have names like “student numbers” and “patient numbers”,
and they almost always turn out to have originally been
a unique record identifier in some database system or other.
As the query below demonstrates,
SQLite [automatically numbers records][rowid] as they’re added to tables,
and we can use those record numbers in queries:</p>


In [ ]:
%%sql
SELECT rowid, * FROM Person;

<blockquote class="question" style="border: 2px solid #8A9AD0; margin: 1em 0.2em">
<h3 id="-icon-question--question-listing-radiation-readings">❓ Question: Listing Radiation Readings</h3>
<p>Write a query that lists all radiation readings from the DR-1 site.</p>
<blockquote class="solution" style="display:none">
<div style="color: #555; font-size: 95%;">Hint: Select the text with your mouse to see the answer</div><h3 id="-icon-solution--solution">👁 Solution</h3>
<div class="language-plaintext highlighter-rouge"><div><pre style="color: inherit; background: transparent"><code style="color: inherit">SELECT Survey.reading
FROM Site JOIN Visited JOIN Survey
ON Site.name = Visited.site
AND Visited.id = Survey.taken
WHERE Site.name = 'DR-1'
AND Survey.quant = 'rad';
</code></pre></div>    </div>
<table>
<thead>
<tr>
<th>reading</th>
</tr>
</thead>
<tbody>
<tr>
<td>9.82</td>
</tr>
<tr>
<td>7.8</td>
</tr>
<tr>
<td>11.25</td>
</tr>
</tbody>
</table>
</blockquote>
</blockquote>


In [ ]:
%%sql
-- Try solutions here!

<blockquote class="question" style="border: 2px solid #8A9AD0; margin: 1em 0.2em">
<h3 id="-icon-question--question-wheres-frank">❓ Question: Where’s Frank?</h3>
<p>Write a query that lists all sites visited by people named “Frank”.</p>
<blockquote class="solution" style="display:none">
<div style="color: #555; font-size: 95%;">Hint: Select the text with your mouse to see the answer</div><h3 id="-icon-solution--solution">👁 Solution</h3>
<div class="language-plaintext highlighter-rouge"><div><pre style="color: inherit; background: transparent"><code style="color: inherit">SELECT DISTINCT Site.name
FROM Site JOIN Visited JOIN Survey JOIN Person
ON Site.name = Visited.site
AND Visited.id = Survey.taken
AND Survey.person = Person.id
WHERE Person.personal = 'Frank';
</code></pre></div>    </div>
<table>
<thead>
<tr>
<th>name</th>
</tr>
</thead>
<tbody>
<tr>
<td>DR-3</td>
</tr>
</tbody>
</table>
</blockquote>
</blockquote>


In [ ]:
%%sql
-- Try solutions here!

<blockquote class="question" style="border: 2px solid #8A9AD0; margin: 1em 0.2em">
<h3 id="-icon-question--question-reading-queries">❓ Question: Reading Queries</h3>
<p>Describe in your own words what the following query produces:</p>
<div class="language-plaintext highlighter-rouge"><div><pre style="color: inherit; background: transparent"><code style="color: inherit">SELECT Site.name FROM Site JOIN Visited
ON Site.lat &lt; -49.0 AND Site.name = Visited.site AND Visited.dated &gt;= '1932-01-01';
</code></pre></div>  </div>
</blockquote>


In [ ]:
%%sql
-- Try solutions here!

<blockquote class="question" style="border: 2px solid #8A9AD0; margin: 1em 0.2em">
<h3 id="-icon-question--question-who-has-been-where">❓ Question: Who Has Been Where?</h3>
<p>Write a query that shows each site with exact location (lat, long) ordered by visited date,
followed by personal name and family name of the person who visited the site
and the type of measurement taken and its reading. Please avoid all null values.
Tip: you should get 15 records with 8 fields.</p>
<blockquote class="solution" style="display:none">
<div style="color: #555; font-size: 95%;">Hint: Select the text with your mouse to see the answer</div><h3 id="-icon-solution--solution">👁 Solution</h3>
<div class="language-plaintext highlighter-rouge"><div><pre style="color: inherit; background: transparent"><code style="color: inherit">SELECT Site.name, Site.lat, Site.long, Person.personal, Person.family, Survey.quant, Survey.reading, Visited.dated
FROM Site JOIN Visited JOIN Survey JOIN Person
ON Site.name = Visited.site
AND Visited.id = Survey.taken
AND Survey.person = Person.id
WHERE Survey.person IS NOT NULL
AND Visited.dated IS NOT NULL
ORDER BY Visited.dated;
</code></pre></div>    </div>
<table>
<thead>
<tr>
<th>name</th>
<th>lat</th>
<th>long</th>
<th>personal</th>
<th>family</th>
<th>quant</th>
<th>reading</th>
<th>dated</th>
</tr>
</thead>
<tbody>
<tr>
<td>DR-1</td>
<td>-49.85</td>
<td>-128.57</td>
<td>William</td>
<td>Dyer</td>
<td>rad</td>
<td>9.82</td>
<td>1927-02-08</td>
</tr>
<tr>
<td>DR-1</td>
<td>-49.85</td>
<td>-128.57</td>
<td>William</td>
<td>Dyer</td>
<td>sal</td>
<td>0.13</td>
<td>1927-02-08</td>
</tr>
<tr>
<td>DR-1</td>
<td>-49.85</td>
<td>-128.57</td>
<td>William</td>
<td>Dyer</td>
<td>rad</td>
<td>7.8</td>
<td>1927-02-10</td>
</tr>
<tr>
<td>DR-1</td>
<td>-49.85</td>
<td>-128.57</td>
<td>William</td>
<td>Dyer</td>
<td>sal</td>
<td>0.09</td>
<td>1927-02-10</td>
</tr>
<tr>
<td>DR-3</td>
<td>-47.15</td>
<td>-126.72</td>
<td>Anderson</td>
<td>Lake</td>
<td>sal</td>
<td>0.05</td>
<td>1930-01-07</td>
</tr>
<tr>
<td>DR-3</td>
<td>-47.15</td>
<td>-126.72</td>
<td>Frank</td>
<td>Pabodie</td>
<td>rad</td>
<td>8.41</td>
<td>1930-01-07</td>
</tr>
<tr>
<td>DR-3</td>
<td>-47.15</td>
<td>-126.72</td>
<td>Frank</td>
<td>Pabodie</td>
<td>temp</td>
<td>-21.5</td>
<td>1930-01-07</td>
</tr>
<tr>
<td>DR-3</td>
<td>-47.15</td>
<td>-126.72</td>
<td>Frank</td>
<td>Pabodie</td>
<td>rad</td>
<td>7.22</td>
<td>1930-01-12</td>
</tr>
<tr>
<td>DR-3</td>
<td>-47.15</td>
<td>-126.72</td>
<td>Anderson</td>
<td>Lake</td>
<td>sal</td>
<td>0.1</td>
<td>1930-02-26</td>
</tr>
<tr>
<td>DR-3</td>
<td>-47.15</td>
<td>-126.72</td>
<td>Frank</td>
<td>Pabodie</td>
<td>rad</td>
<td>4.35</td>
<td>1930-02-26</td>
</tr>
<tr>
<td>DR-3</td>
<td>-47.15</td>
<td>-126.72</td>
<td>Frank</td>
<td>Pabodie</td>
<td>temp</td>
<td>-18.5</td>
<td>1930-02-26</td>
</tr>
<tr>
<td>MSK-4</td>
<td>-48.87</td>
<td>-123.4</td>
<td>Anderson</td>
<td>Lake</td>
<td>rad</td>
<td>1.46</td>
<td>1932-01-14</td>
</tr>
<tr>
<td>MSK-4</td>
<td>-48.87</td>
<td>-123.4</td>
<td>Anderson</td>
<td>Lake</td>
<td>sal</td>
<td>0.21</td>
<td>1932-01-14</td>
</tr>
<tr>
<td>MSK-4</td>
<td>-48.87</td>
<td>-123.4</td>
<td>Valentina</td>
<td>Roerich</td>
<td>sal</td>
<td>22.5</td>
<td>1932-01-14</td>
</tr>
<tr>
<td>DR-1</td>
<td>-49.85</td>
<td>-128.57</td>
<td>Valentina</td>
<td>Roerich</td>
<td>rad</td>
<td>11.25</td>
<td>1932-03-22</td>
</tr>
</tbody>
</table>
</blockquote>
</blockquote>


In [ ]:
%%sql
-- Try solutions here!

<p>A good visual explanation of joins can be found <a href="https://sql-joins.leopard.in.ua/">here</a></p>
<h1 id="data-hygiene">Data Hygiene</h1>
<p>Now that we have seen how joins work, we can see why the relational
model is so useful and how best to use it.  The first rule is that
every value should be atomic, i.e., not
contain parts that we might want to work with separately.  We store
personal and family names in separate columns instead of putting the
entire name in one column so that we don’t have to use substring
operations to get the name’s components.  More importantly, we store
the two parts of the name separately because splitting on spaces is
unreliable: just think of a name like “Eloise St. Cyr” or “Jan Mikkel
Steubart”.</p>
<p>The second rule is that every record should have a unique primary key.
This can be a serial number that has no intrinsic meaning,
one of the values in the record (like the <code style="color: inherit">id</code> field in the <code style="color: inherit">Person</code> table),
or even a combination of values:
the triple <code style="color: inherit">(taken, person, quant)</code> from the <code style="color: inherit">Survey</code> table uniquely identifies every measurement.</p>
<p>The third rule is that there should be no redundant information.
For example,
we could get rid of the <code style="color: inherit">Site</code> table and rewrite the <code style="color: inherit">Visited</code> table like this:</p>
<table>
<thead>
<tr>
<th>id</th>
<th>lat</th>
<th>long</th>
<th>dated</th>
</tr>
</thead>
<tbody>
<tr>
<td>619</td>
<td>-49.85</td>
<td>-128.57</td>
<td>1927-02-08</td>
</tr>
<tr>
<td>622</td>
<td>-49.85</td>
<td>-128.57</td>
<td>1927-02-10</td>
</tr>
<tr>
<td>734</td>
<td>-47.15</td>
<td>-126.72</td>
<td>1930-01-07</td>
</tr>
<tr>
<td>735</td>
<td>-47.15</td>
<td>-126.72</td>
<td>1930-01-12</td>
</tr>
<tr>
<td>751</td>
<td>-47.15</td>
<td>-126.72</td>
<td>1930-02-26</td>
</tr>
<tr>
<td>752</td>
<td>-47.15</td>
<td>-126.72</td>
<td>None</td>
</tr>
<tr>
<td>837</td>
<td>-48.87</td>
<td>-123.40</td>
<td>1932-01-14</td>
</tr>
<tr>
<td>844</td>
<td>-49.85</td>
<td>-128.57</td>
<td>1932-03-22</td>
</tr>
</tbody>
</table>
<p>In fact,
we could use a single table that recorded all the information about each reading in each row,
just as a spreadsheet would.
The problem is that it’s very hard to keep data organized this way consistent:
if we realize that the date of a particular visit to a particular site is wrong,
we have to change multiple records in the database.
What’s worse,
we may have to guess which records to change,
since other sites may also have been visited on that date.</p>
<p>The fourth rule is that the units for every value should be stored explicitly.
Our database doesn’t do this,
and that’s a problem:
Roerich’s salinity measurements are several orders of magnitude larger than anyone else’s,
but we don’t know if that means she was using parts per million instead of parts per thousand,
or whether there actually was a saline anomaly at that site in 1932.</p>
<p>Stepping back,
data and the tools used to store it have a symbiotic relationship:
we use tables and joins because it’s efficient,
provided our data is organized a certain way,
but organize our data that way because we have tools to manipulate it efficiently.
As anthropologists say,
the tool shapes the hand that shapes the tool.</p>
<blockquote class="question" style="border: 2px solid #8A9AD0; margin: 1em 0.2em">
<h3 id="-icon-question--question-identifying-atomic-values">❓ Question: Identifying Atomic Values</h3>
<p>Which of the following are atomic values? Which are not? Why?</p>
<ul>
<li>New Zealand</li>
<li>87 Turing Avenue</li>
<li>January 25, 1971</li>
<li>the XY coordinate (0.5, 3.3)</li>
</ul>
<blockquote class="solution" style="display:none">
<div style="color: #555; font-size: 95%;">Hint: Select the text with your mouse to see the answer</div><h3 id="-icon-solution--solution">👁 Solution</h3>
<p>New Zealand is the only clear-cut atomic value.</p>
<p>The address and the XY coordinate contain more than one piece of information
which should be stored separately:</p>
<ul>
<li>House number, street name</li>
<li>X coordinate, Y coordinate</li>
</ul>
<p>The date entry is less clear cut, because it contains month, day, and year elements.
However, there is a <code style="color: inherit">DATE</code> datatype in SQL, and dates should be stored using this format.
If we need to work with the month, day, or year separately, we can use the SQL functions available for our database software
(for example <a href="https://docs.oracle.com/cd/B19306_01/server.102/b14200/functions050.htm"><code class="language-plaintext highlighter-rouge">EXTRACT</code></a> or <a href="http://www.sqlite.org/lang_datefunc.html"><code class="language-plaintext highlighter-rouge">STRFTIME</code></a> for SQLite).</p>
</blockquote>
</blockquote>


In [ ]:
%%sql
-- Try solutions here!

<blockquote class="question" style="border: 2px solid #8A9AD0; margin: 1em 0.2em">
<h3 id="-icon-question--question-identifying-a-primary-key">❓ Question: Identifying a Primary Key</h3>
<p>What is the primary key in this table?
I.e., what value or combination of values uniquely identifies a record?</p>
<table>
<thead>
<tr>
<th>latitude</th>
<th>longitude</th>
<th>date</th>
<th>temperature</th>
</tr>
</thead>
<tbody>
<tr>
<td>57.3</td>
<td>-22.5</td>
<td>2015-01-09</td>
<td>-14.2</td>
</tr>
</tbody>
</table>
<blockquote class="solution" style="display:none">
<div style="color: #555; font-size: 95%;">Hint: Select the text with your mouse to see the answer</div><h3 id="-icon-solution--solution">👁 Solution</h3>
<p>Latitude, longitude, and date are all required to uniquely identify the temperature record.</p>
</blockquote>
</blockquote>


In [ ]:
%%sql
-- Try solutions here!

<h1 id="creating-and-modifying-data">Creating and Modifying Data</h1>
<p>So far we have only looked at how to get information out of a database,
both because that is more frequent than adding information,
and because most other operations only make sense
once queries are understood.
If we want to create and modify data,
we need to know two other sets of commands.</p>
<p>The first pair are [<code class="language-plaintext highlighter-rouge">CREATE TABLE</code>][create-table] and [<code class="language-plaintext highlighter-rouge">DROP TABLE</code>][drop-table].
While they are written as two words,
they are actually single commands.
The first one creates a new table;
its arguments are the names and types of the table’s columns.
For example,
the following statements create the four tables in our survey database:</p>


In [ ]:
%%sql
CREATE TABLE Person(id text, personal text, family text);
CREATE TABLE Site(name text, lat real, long real);
CREATE TABLE Visited(id integer, site text, dated text);
CREATE TABLE Survey(taken integer, person text, quant text, reading real);

<p>We can get rid of one of our tables using:</p>


In [ ]:
%%sql
DROP TABLE Survey;

<p>Be very careful when doing this:
if you drop the wrong table, hope that the person maintaining the database has a backup,
but it’s better not to have to rely on it.</p>
<p>Different database systems support different data types for table columns,
but most provide the following:</p>
<table>
<thead>
<tr>
<th>data type</th>
<th>use</th>
</tr>
</thead>
<tbody>
<tr>
<td>INTEGER</td>
<td>a signed integer</td>
</tr>
<tr>
<td>REAL</td>
<td>a floating point number</td>
</tr>
<tr>
<td>TEXT</td>
<td>a character string</td>
</tr>
<tr>
<td>BLOB</td>
<td>a “binary large object”, such as an image</td>
</tr>
</tbody>
</table>
<p>Most databases also support Booleans and date/time values;
SQLite uses the integers 0 and 1 for the former,
and represents the latter as text or numeric fields.</p>
<p>An increasing number of databases also support geographic data types,
such as latitude and longitude.
Keeping track of what particular systems do or do not offer,
and what names they give different data types,
is an unending portability headache.</p>
<blockquote class="tip" style="border: 2px solid #FFE19E; margin: 1em 0.2em">
<h3 id="-icon-tip--which-database-should-i-use">💡 Which database should I use?</h3>
<p>SQLite is fantastic for small databases or embedded into applications where
you want to be able to use SQL to query and process data.</p>
<p>However for any real analysis PostgreSQL is usually the best choice, it
scales incredibly well and can meet a wide range of use cases. It has good
data type support.</p>
</blockquote>
<blockquote class="tip" style="border: 2px solid #FFE19E; margin: 1em 0.2em">
<h3 id="-icon-tip--do-you-have-geographic-data">💡 Do you have geographic data?</h3>
<p>Use Postgres. The <a href="https://postgis.net/">PostGIS</a> library is fantastic and industry standard for storing geographic data in a database.</p>
</blockquote>
<p>When we create a table,
we can specify several kinds of constraints on its columns.
For example,
a better definition for the <code style="color: inherit">Survey</code> table would be:</p>


In [ ]:
%%sql
CREATE TABLE Survey(
    taken   integer not null, -- where reading taken
    person  text,             -- may not know who took it
    quant   text not null,    -- the quantity measured
    reading real not null,    -- the actual reading
    primary key(taken, quant),
    foreign key(taken) references Visited(id),
    foreign key(person) references Person(id)
);

<p>Once again,
exactly what constraints are available
and what they’re called
depends on which database manager we are using.</p>
<p>Once tables have been created,
we can add, change, and remove records using our other set of commands,
<code class="language-plaintext highlighter-rouge">INSERT</code>, <code style="color: inherit">UPDATE</code>, and <code style="color: inherit">DELETE</code>.</p>
<p>Here is an example of inserting rows into the <code style="color: inherit">Site</code> table:</p>


In [ ]:
%%sql
INSERT INTO Site (name, lat, long) VALUES ('DR-1', -49.85, -128.57);
INSERT INTO Site (name, lat, long) VALUES ('DR-3', -47.15, -126.72);
INSERT INTO Site (name, lat, long) VALUES ('MSK-4', -48.87, -123.40);

<p>We can also insert values into one table directly from another:</p>


In [ ]:
%%sql
CREATE TABLE JustLatLong(lat real, long real);
INSERT INTO JustLatLong SELECT lat, long FROM Site;

<p>Modifying existing records is done using the <code style="color: inherit">UPDATE</code> statement.
To do this we tell the database which table we want to update,
what we want to change the values to for any or all of the fields,
and under what conditions we should update the values.</p>
<p>For example, if we made a mistake when entering the lat and long values
of the last <code style="color: inherit">INSERT</code> statement above, we can correct it with an update:</p>


In [ ]:
%%sql
UPDATE Site SET lat = -47.87, long = -122.40 WHERE name = 'MSK-4';

<p>Be careful to not forget the <code style="color: inherit">WHERE</code> clause or the update statement will
modify <em>all</em> of the records in the database.</p>
<p>Deleting records can be a bit trickier,
because we have to ensure that the database remains internally consistent.
If all we care about is a single table,
we can use the <code style="color: inherit">DELETE</code> command with a <code style="color: inherit">WHERE</code> clause
that matches the records we want to discard.
For example,
once we realize that Frank Danforth didn’t take any measurements,
we can remove him from the <code style="color: inherit">Person</code> table like this:</p>


In [ ]:
%%sql
DELETE FROM Person WHERE id = 'danforth';

<p>But what if we removed Anderson Lake instead?
Our <code style="color: inherit">Survey</code> table would still contain seven records
of measurements he’d taken,
but that’s never supposed to happen:
<code style="color: inherit">Survey.person</code> is a foreign key into the <code style="color: inherit">Person</code> table,
and all our queries assume there will be a row in the latter
matching every value in the former.</p>
<p>This problem is called referential integrity:
we need to ensure that all references between tables can always be resolved correctly.
One way to do this is to delete all the records
that use <code style="color: inherit">'lake'</code> as a foreign key
before deleting the record that uses it as a primary key.
If our database manager supports it,
we can automate this
using cascading delete.
However,
this technique is outside the scope of this chapter.</p>
<blockquote class="tip" style="border: 2px solid #FFE19E; margin: 1em 0.2em">
<h3 id="-icon-tip--hybrid-storage-models">💡 Hybrid Storage Models</h3>
<p>Many applications use a hybrid storage model
instead of putting everything into a database:
the actual data (such as astronomical images) is stored in files,
while the database stores the files’ names,
their modification dates,
the region of the sky they cover,
their spectral characteristics,
and so on.
This is also how most music player software is built:
the database inside the application keeps track of the MP3 files,
but the files themselves live on disk.</p>
</blockquote>
<blockquote class="question" style="border: 2px solid #8A9AD0; margin: 1em 0.2em">
<h3 id="-icon-question--question-replacing-null">❓ Question: Replacing NULL</h3>
<p>Write an SQL statement to replace all uses of <code style="color: inherit">null</code> in
<code style="color: inherit">Survey.person</code> with the string <code style="color: inherit">'unknown'</code>.</p>
<blockquote class="solution" style="display:none">
<div style="color: #555; font-size: 95%;">Hint: Select the text with your mouse to see the answer</div><h3 id="-icon-solution--solution">👁 Solution</h3>
<div class="language-plaintext highlighter-rouge"><div><pre style="color: inherit; background: transparent"><code style="color: inherit">UPDATE Survey SET person = 'unknown' WHERE person IS NULL;
</code></pre></div>    </div>
</blockquote>
</blockquote>


In [ ]:
%%sql
-- Try solutions here!

<blockquote class="question" style="border: 2px solid #8A9AD0; margin: 1em 0.2em">
<h3 id="-icon-question--question-backing-up-with-sql">❓ Question: Backing Up with SQL</h3>
<p>SQLite has several administrative commands that aren’t part of the
SQL standard.  One of them is <code style="color: inherit">.dump</code>, which prints the SQL commands
needed to re-create the database.  Another is <code style="color: inherit">.read</code>, which reads a
file created by <code style="color: inherit">.dump</code> and restores the database.  A colleague of
yours thinks that storing dump files (which are text) in version
control is a good way to track and manage changes to the database.
What are the pros and cons of this approach?  (Hint: records aren’t
stored in any particular order.)</p>
<blockquote class="solution" style="display:none">
<div style="color: #555; font-size: 95%;">Hint: Select the text with your mouse to see the answer</div><h3 id="-icon-solution--solution">👁 Solution</h3>
<h4 id="advantages">Advantages</h4>
<ul>
<li>A version control system will be able to show differences between versions
of the dump file; something it can’t do for binary files like databases</li>
<li>A VCS only saves changes between versions, rather than a complete copy of
each version (save disk space)</li>
<li>The version control log will explain the reason for the changes in each version
of the database</li>
</ul>
<h4 id="disadvantages">Disadvantages</h4>
<ul>
<li>Artificial differences between commits because records don’t have a fixed order</li>
</ul>
</blockquote>
</blockquote>


In [ ]:
%%sql
-- Try solutions here!

# Key Points

- Use aggregation functions to combine multiple values.
- Aggregation functions ignore `null` values.
- Aggregation happens after filtering.
- Use GROUP BY to combine subsets separately.
- If no aggregation function is specified for a field, the query may return an arbitrary value for that field.
- Use JOIN to combine data from two tables.
- Use table.field notation to refer to fields when doing joins.
- Every fact should be represented in a database exactly once.
- A join produces all combinations of records from one table with records from another.
- A primary key is a field (or set of fields) whose values uniquely identify the records in a table.
- A foreign key is a field (or set of fields) in one table whose values are a primary key in another table.
- We can eliminate meaningless combinations of records by matching primary keys and foreign keys between tables.
- The most common join condition is matching keys.
- Every value in a database should be atomic.
- Every record should have a unique primary key.
- A database should not contain redundant information.
- Units and similar metadata should be stored with the data.
- Use CREATE and DROP to create and delete tables.
- Use INSERT to add data.
- Use UPDATE to modify existing data.
- Use DELETE to remove data.
- It is simpler and safer to modify data when every record has a unique primary key.
- Do not create dangling references by deleting records that other records refer to.
- General-purpose languages have libraries for accessing databases.
- To connect to a database, a program must use a library specific to that database manager.
- These libraries use a connection-and-cursor model.
- Programs can read query results in batches or all at once.
- Queries should be written using parameter substitution, not string formatting.

# Congratulations on successfully completing this tutorial!

Please [fill out the feedback on the GTN website](https://training.galaxyproject.org/training-material/topics/data-science/tutorials/sql-advanced/tutorial.html#feedback) and check there for further resources!
